In [1]:
import os
import openai
import csv
from dotenv import load_dotenv
from openai import OpenAI
#from google.colab import userdata

In [ ]:
# Environment

#SYSTEM_PROMPT = "You are a helpful assistant that collects data from emails."

#API_KEY = ""

# using Collab Secrets
#API_KEY=userdata.get('openai'))

# using .env
load_dotenv()
#print(os.getenv('SECRET'))
API_KEY = os.getenv('API_KEY')

# Initialize the OpenAI client with your API key
client = OpenAI(api_key=API_KEY)

In [2]:
# Define the directory and output CSV file
input_folder = "data"
output_csv = "data/data.csv"

In [3]:
# Function to read HTML file
def read_html_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

#html_content = read_html_file('test/mailexp_20250114-_noreply@deutschebahn.com_-1323.html')
#print(html_content)

In [4]:
### WORKS !
# # Function to save data to CSV
### ggf newline ändern?

def save_to_csv(data, output_file):
    header = ["date", "departure_time", "departure_station", "arrival_time", "arrival_station", "price", "booking_timestamp", "order_number"]
    with open(output_file, 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(header)
        writer.writerows(data)

In [15]:
### WORKS !
# # Function to save extracted data to CSV

#def save_to_csv(data, output_file):
#    with open(output_file, "w", newline="", encoding="utf-8") as csvfile:
#        writer = csv.writer(csvfile)
#        writer.writerow(["date", "departure_time", "departure_station", 
#                         "arrival_time", "arrival_station", "price", 
#                         "booking_timestamp", "order_number"])
#
#        for row in data:
#            writer.writerow(row)

In [16]:
# Function to process HTML content with AI
def extract_email_info(html_content):
    prompt = """Here is a booking confirmation from Deutsche Bahn. 
    Please arrange the following key information in a comma-separated table 
    with the header: date, departure_time, departure_station, arrival_time, 
    arrival_station, price, booking_timestamp, order_number"""
    
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": html_content}
        ],
        temperature=0.1,  # Set to 0 for a deterministic answer
        max_tokens=2000   # adjust
    )
    
    return response.choices[0].message.content.strip()

In [ ]:
# Test
#extract_email_info(html_content)

In [17]:
# Main Execution
email_data = []

# Process each HTML email file
for filename in os.listdir(input_folder):
    if filename.endswith(".html"):
        file_path = os.path.join(input_folder, filename)
        html_content = read_html_file(file_path)
        extracted_text = extract_email_info(html_content)

        # Process AI response (Assuming CSV-like output)
        lines = extracted_text.split("\n")
        for line in lines:
            parts = line.split(",")  # Assuming AI returns comma-separated values
            if len(parts) == 8:  # Ensure all columns are present
                email_data.append(parts)

In [ ]:
# Print
#print(email_data)

In [20]:
# Save extracted data to CSV
save_to_csv(email_data, output_csv)

print(f"CSV saved to {output_csv}")

CSV saved to test/extracted_booking_info.csv


In [ ]:
# Print the model's reply
#print("Response:", response.choices[0].message.content.strip())

# Print token usage details
#print("Tokens Used:", response.usage)